In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [30]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report,average_precision_score,precision_score,f1_score

In [2]:
sample_submission=pd.read_csv("../input/music-genre/dataset/sample_submission.csv")
sample_submission.head()


In [43]:
train=pd.read_csv("../input/music-genre/dataset/train.csv")
train.head()

In [44]:
test=pd.read_csv("../input/music-genre/dataset/test.csv")
test.head()


In [22]:
#df = pd.concat([train, test], axis= 0)
df.isnull().sum()

In [25]:
df.music_genre.value_counts()

In [20]:
df.info()

In [45]:
df.voice_gender=df.voice_gender.fillna('N_a')
df['mode']=df['mode'].replace({'?':"Major"})
df.musician_category=df.musician_category.fillna('N_A')

In [19]:
df.voice_gender=df.voice_gender.astype('category').cat.codes
df['mode']=df['mode'].astype('category').cat.codes
df.musician_category=df.musician_category.astype('category').cat.codes
df['key']=df['key'].astype('category').cat.codes
df.head()

In [46]:
df.popularity = df.popularity.fillna(value=df.popularity.median(), axis=0)
df.danceability = df.danceability.fillna(value=df.key.median(), axis=0)
df.instrumentalness = df.instrumentalness.fillna(value=df.instrumentalness.median(), axis=0)
df.duration_ms=df.duration_ms.fillna(value=df.duration_ms.median(), axis=0)
df.energy=df.energy.fillna(value=df.duration_ms.median(), axis=0)
df.loudness=df.loudness.fillna(value=df.duration_ms.median(), axis=0)
df.speechiness=df.speechiness.fillna(value=df.duration_ms.median(), axis=0)
df.valence=df.valence.fillna(value=df.duration_ms.median(), axis=0)
df.head()

In [47]:
cols=[col for col in df.columns if col not in ['instance_id','track_name','tempo','music_genre']]
cols

In [53]:
#df1=df[cols]
df1.head()

In [51]:
#df1['song_in_min']= df1['duration_ms']/60000
df1['pops']= df1.popularity + df1.danceability + df1.energy  + df1.loudness + df1.speechiness + df1.acousticness+ df1.instrumentalness + df1.liveness + df1.valence 


In [54]:
train1 = df1[:15681]
test1 = df1[15681:]

In [55]:
y=LabelEncoder()
train.music_genre=y.fit_transform(train.music_genre)
y1=train.music_genre

In [58]:
n_folds = 5
subbed = []
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)


for fold, (train_idx, test_idx) in enumerate(kf.split(train1, y1)):
    print('=============== Fold No:',fold+1,'===============')
    X_tr, X_tst = train1.iloc[train_idx], train1.iloc[test_idx]
    y_tr, y_tst = y1.iloc[train_idx], y1.iloc[test_idx]
    
    model = CatBoostClassifier(n_estimators=10000, random_state=42, eval_metric= 'MultiClass')
    model.fit(X_tr, y_tr,eval_set=[(X_tst, y_tst)], early_stopping_rounds=30, verbose=500)
    print(f1_score(y_tst, model.predict(X_tst),average ='macro'))
    subbed.append(f1_score(y_tst, model.predict(X_tst),average ='macro'))
    pred = model.predict(test1)
print(np.mean(subbed))


In [59]:
test['music_genre']=pred
test['music_genre']=y.inverse_transform(test['music_genre'])
test[['instance_id','music_genre']].to_csv('submit001.csv',index=False)
#print (test['credit_card_default'].sum())